In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
encounters = pd.read_csv("encounters.csv")
organizations = pd.read_csv("organizations.csv")
patients = pd.read_csv("patients.csv")
payers = pd.read_csv("payers.csv")
procedures = pd.read_csv("procedures.csv")

In [ ]:
encounters.dtypes

In [ ]:
patients

In [ ]:
patients.dtypes

In [ ]:
patients.head(2)

In [ ]:
encounters.head(2)

In [ ]:
encounters.rename(columns={"PATIENT":"patient_id"}, inplace=True)

In [ ]:
encounters.head(2)

In [ ]:
patients.rename(columns={"Id":"patient_id"}, inplace=True)

In [ ]:
patients.head(2)

In [ ]:
merged_dataset = pd.merge(encounters, patients, on="patient_id", how="left")
merged_dataset

In [ ]:
payers

In [ ]:
merged_dataset.rename(columns={"PAYER":"payers_id"},inplace=True)

In [ ]:
payers.rename(columns={"Id":"payers_id"},inplace=True)

In [ ]:
payers

In [ ]:
merged_dataset = pd.merge(merged_dataset, payers, on="payers_id", how="inner")

In [ ]:
merged_dataset.columns


In [ ]:
procedures

In [ ]:
encounters.rename(columns={"Id":"encounter_id"},inplace=True)

In [ ]:
procedures.rename(columns={"ENCOUNTER":"encounter_id"},inplace=True)

In [ ]:
merged_dataset.rename(columns={"Id":"encounter_id"},inplace=True)

In [ ]:
merged_dataset.columns

In [ ]:
merged_dataset1 = pd.merge(merged_dataset, procedures,on="encounter_id", how="left")
merged_dataset1

In [ ]:
procedures.columns

In [ ]:
merged_dataset1

In [ ]:
merged_dataset1.columns

In [ ]:
merged_dataset1.isnull().sum()

In [ ]:
merged_dataset1.dtypes

In [ ]:
merged_dataset1["BIRTHDATE"] = pd.to_datetime(merged_dataset1["BIRTHDATE"], errors="coerce")

In [ ]:
merged_dataset1

In [ ]:
merged_dataset1.dtypes

In [ ]:
merged_dataset1.columns

In [ ]:
merged_dataset1.shape

# Healthcare Utilization by Age Group

In [ ]:
from datetime import datetime
current_date = datetime.now().date()
ages = []
for birthdate in merged_dataset1["BIRTHDATE"]:
    age = current_date.year - birthdate.year
    if(current_date.month, current_date.day) < (birthdate.month, birthdate.day):
        age-=1
    ages.append(age)
merged_dataset1["AGE"] = ages
merged_dataset1[["BIRTHDATE","AGE"]].head()

In [ ]:
def age_calculate(age):
    if age <= 18:
        return "0-18"
    elif age <= 35:
        return "19-35"
    elif age <= 50:
        return "36-50"
    elif age <= 65:
        return "51-65"
    else:
        return "66+"
merged_dataset1["AGE_GROUP"] = [age_calculate(age) for age in merged_dataset1["AGE"]]

In [ ]:
age_group_counts = merged_dataset1.groupby("AGE_GROUP")["encounter_id"].nunique()
print(age_group_counts)

In [ ]:
plt.figure(figsize=(12, 6))
bars = plt.bar(age_group_counts.index, age_group_counts.values, color="skyblue") 
plt.bar_label(bars)
plt.title("Healthcare Utilization by Age Group", fontsize=16)
plt.xlabel("Age Group", fontsize=14)
plt.ylabel("Number of Unique Encounters", fontsize=14)
plt.xticks(rotation=0)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# Average Cost of Encounters by Type

In [ ]:
merged_dataset1.columns

In [ ]:
average_cost = merged_dataset1["TOTAL_CLAIM_COST"].mean()
average_cost

In [ ]:
average_cost_by_encounters = merged_dataset1.groupby("ENCOUNTERCLASS")["TOTAL_CLAIM_COST"].mean()
plt.figure(figsize=(10,5))
bars = plt.bar(average_cost_by_encounters.index, average_cost_by_encounters.values, color="lightgreen")
plt.bar_label(bars)
plt.title("Average Cost By Encounter Type")
plt.xlabel("Encounter Type")
plt.ylabel("Average Cost")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.plot()